## Loading events

In [1]:
from CCA import DataProcessor

# Change to your own path to the base of the dataset 
# e.g. to get event1 located in x/y/z/event1, path_to_base = "x/y/z"
path_to_base = None

event_id = "event000001000"

dp = DataProcessor(path_to_base)

# Loading event(s)
dp.load([event_id])

In [2]:
# Checking the loaded raw data 

event = dp.raw_data[event_id]

print(list(event.data.keys()))

['cells', 'hits', 'particles', 'truth']


In [4]:
event.data["particles"]

,particle_id,vx,vy,vz,px,py,pz,q,nhits
0,4503668346847232,-0.009288,0.009861,-0.077879,-0.055269,0.323272,-0.203492,-1,8
1,4503737066323968,-0.009288,0.009861,-0.077879,-0.948125,0.470892,2.010060,1,11
2,4503805785800704,-0.009288,0.009861,-0.077879,-0.886484,0.105749,0.683881,-1,0
3,4503874505277440,-0.009288,0.009861,-0.077879,0.257539,-0.676718,0.991616,1,12
4,4503943224754176,-0.009288,0.009861,-0.077879,16.439400,-15.548900,-39.824900,1,3
...,...,...,...,...,...,...,...,...,...
12258,968305530860736513,-171.641000,302.668000,-1220.000000,-0.110735,-0.058300,-0.137469,1,0
12259,968305530860740610,-171.641000,302.668000,-1220.000000,0.000191,-0.169234,-0.248553,1,4
12260,968306149319245824,33.448100,20.703400,-102.515000,0.486316,0.268579,-1.227300,1,10
12261,968306218038722560,33.448100,20.703400,-102.515000,0.082110,0.084122,-0.415145,-1,1


## Processing data from an event

In [5]:
pT_min = 1.0

# default values
n_particles = "all"
remove_layer_duplicates = True
volume_ids = [8, 13, 17]


hits, truth = dp.process(event_id, pT_min, n_particles, remove_layer_duplicates, volume_ids)


In [6]:
hits

,hit_id,x,y,z,volume_id,layer_id,module_id
16945,16946,-31.144400,-7.68151,-284.714,8,802,33
16950,16951,-24.658900,-20.33090,-315.132,8,802,34
16970,16971,-0.557692,-31.95380,-311.788,8,802,36
16976,16977,-4.841760,-31.71210,-321.638,8,802,36
16986,16987,10.351200,-30.21100,-315.857,8,802,37
...,...,...,...,...,...,...,...
117905,117906,1019.120000,58.54800,1057.000,17,1704,3118
117920,117921,858.422000,559.66900,1057.000,17,1704,3130
117943,117944,378.553000,951.19600,1067.800,17,1704,3145
118001,118002,-993.280000,235.12300,1035.400,17,1704,3187


## Running the algorithm

In [7]:
from CCA import CCAProcessor, CellsProcessor, NeighboursProcessor, EvolutionProcessor, TracksProcessor

In [8]:
# Running the entire algorithm

min_cells_angle = 0.4
max_neigh_angle = 0.15
min_track_length = 4

CCAProc = CCAProcessor(hits)
CCAProc.runAll(min_cells_angle, max_neigh_angle, min_track_length)

Forming pairs...
55635 cells formed.

Finding Neighbours...


/Users/archibaldruban/Google Drive/1. Education/UCL/1. BSc Theoretical Physics/_year 4/Master's Project/code/CCA/data/hits.py:37: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._lids = self["layer_id"].drop_duplicates().to_list()


82818 neighbours found.

Evolving CA...
Evolution completed.

Generating track candidates...
1169704 tracks generated.


In [9]:
# Alternatively, can run individual parts


    # forming cells
# CCAProc.formCells()
# or 
# cellsProc = CellsProcessor(hits, min_cells_angle)
# cellsProc.form()


    # Finding Neighbours
# CCAProc.findNeighbours()
# or
# neighboursProc = NeighboursProcessor(cellsProc, max_neigh_angle)
# neighboursProc.find_neighbours()


    # Evolving the CA
# CCAProc.evolve()
# or
# evolutionProc = EvolutionProcessor(neighboursProc)
# evolutionProc.evolve()


    # Generating track candidates
# CCAProc.generateTracks()
# or
# tracksProc = TracksProcessor(evolutionProc, min_track_length)
# tracksProc.generate()

## Evaluating the algorithm's performance 

In [10]:
from CCA import CCAEvaluator

In [12]:
# To evaluate cells, neighbours & tracks

CCAEval = CCAEvaluator(CCAProc, truth)
CCAEval.evaluate()


/Users/archibaldruban/Google Drive/1. Education/UCL/1. BSc Theoretical Physics/_year 4/Master's Project/code/CCA/data/truth.py:74: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._dict = true_tracks_dict


In [13]:
CCAEval.performance

,efficiency,purity
cells,0.969487,0.093089
neighbours,0.952804,0.050460
tracks,0.651341,0.000436


In [ ]:
# Alternatively, can evaluate individual parts using CellsEvaluator, NeighboursEvaluator, TracksEvaluator
# and by calling their respective "evaluate()" method and then printing their "performance" property
# from CCA import CellsEvaluator, NeighboursEvaluator, TracksEvaluator